## Build a Baseline Retrieval Augment Generation Solution
In this notebook we will build an initial solution that will utilize a pre-trained model augmented with a contextual data from a vector store retriever. At a high level, the solution will work as follows:
- Based on a user's query, we will retrieve the top-k most similar documents from the vector store.
- Provide the relevant documents as part of the prompt to the model along with the user's question
- Generate the answer using the model

![Basic RAG](images/chatbot_lang.png)

We'll evaluate several aspects of the solution including:
- The accuracy of the retrieved context
- The quality of the generated answer

These metrics will help determine whether a solution using purely pre-trained models is viable or whether we need to consider more complex strategies or fine-tuning

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "amazon.titan-embed-text-v2:0",
    "mistral.mixtral-8x7b-instruct-v0:1",
    "mistral.mistral-7b-instruct-v0:2",
    "us.amazon.nova-lite-v1:0"
]
validate_model_access(required_models)

### Data Ingestion
The prepared datasets have been split into training and validation sets. We will load documents associated with both sets into a vector store for retrieval.

In [ ]:
from pathlib import Path
from itertools import chain
from rich import print as rprint
from IPython.display import display, Markdown
import json
from concurrent.futures import ThreadPoolExecutor
import langchain
from langchain_core.documents import Document
from langchain_aws.chat_models import ChatBedrockConverse
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
import boto3

import pickle
from io import BytesIO
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import asyncio
import nest_asyncio
nest_asyncio.apply()


data_path = Path("data/prepared_data")
train_data = (data_path / "prepared_data_train.jsonl").read_text().splitlines()
test_data = (data_path / "prepared_data_test.jsonl").read_text().splitlines()

doc_ids = []
documents = []

# Create a list of LangChain documents that can then be used to ingest into a vector store

for record in chain(train_data, test_data):
    json_record = json.loads(record)
    if json_record["ref_doc_id"] not in doc_ids:
        doc_ids.append(json_record["ref_doc_id"])
        doc = Document(page_content=json_record["context"], metadata=json_record["section_metadata"])
        documents.append(doc)

print(f"Loaded {len(documents)} sections")

In [ ]:
import mlflow_utils
import mlflow

mlflow_config_path = Path("mlflow_config.json")
if not mlflow_config_path.exists():
    rprint(
        "No MLFlow configuration found. Please run the first notebook to set up MLFlow."
    )
else:
    mlflow_config = json.loads(mlflow_config_path.read_text())
    server_status = mlflow_utils.check_server_status(
        mlflow_config["tracking_server_name"]
    )
    if server_status["IsActive"] == "Active":
        rprint(
            f'MLFlow server is available. The current status is: {server_status["TrackingServerStatus"]}'
        )
        mlflow_available = True
        mlflow.set_tracking_uri(mlflow_config["tracking_server_arn"])
    else:
        mlflow_available = False
        rprint(
            f'MLFlow server is not available. The current status is: {server_status["TrackingServerStatus"]}'
        )

Next we will initialize the embedding model that will be used to vectorize the documents and queries. We will use the `amazon.titan-embed-text-v2:0` model for this purpose.

In [ ]:
boto3_session=boto3.session.Session()

bedrock_runtime = boto3_session.client("bedrock-runtime")
bedrock_client = boto3_session.client("bedrock")

embedding_modelId = "amazon.titan-embed-text-v2:0"

embed_model = BedrockEmbeddings(
    model_id=embedding_modelId,
    model_kwargs={"dimensions": 1024, "normalize": True},
    client=bedrock_runtime,
)

query = "Do I really need to fine-tune the large language models?"
response = embed_model.embed_query(query)
rprint(f"Generated an embedding with {len(response)} dimensions. Sample of first 10 dimensions:\n", response[:10])

The documents can now be ingested into a vector store. We will utilize a local vector store backed by the `faiss` library for this purpose. In production scenarios, a more scalable solution like OpenSearch or pgvector should be used.

In [ ]:
vector_store_file = "baseline_rag_vec_db.pkl"

if not Path(vector_store_file).exists():
    rprint(f"Vector store file {vector_store_file} does not exist. Will create a new vector store.")
    CREATE_NEW = True
else:
    rprint(f"Vector store file {vector_store_file} already exists and will be reused. Delete it or change the file name above to if you wish to create a new vector store.")
    CREATE_NEW = False 

if CREATE_NEW:
    vec_db = FAISS.from_documents(documents, embed_model)
    pickle.dump(vec_db.serialize_to_bytes(), open(vector_store_file, "wb"))
    
else:
    if not Path(vector_store_file).exists():
        raise FileNotFoundError(f"Vector store file {vector_store_file} not found. Set CREATE_NEW to True to create a new vector store.")
    
    vector_db_buff = BytesIO(pickle.load(open(vector_store_file, "rb")))
    vec_db = FAISS.deserialize_from_bytes(serialized=vector_db_buff.read(), embeddings=embed_model, allow_dangerous_deserialization=True)

### Evaluate the retrieval performance
Before moving on to the generation step, we should validate the performance of the retriever. The large language model will not be able to generate accurate answers if the retrieved context is not relevant. We will evaluate the retriever using the validation set. The prepared validation set contains 400 questions along with relevant contexts. For each question, we have the unique document id of the relevant context. So our evaluation is simple: we will retrieve the top-k documents for each question and check if the relevant context is present in the top-k results. We will then calculate the recall or Hit Rate of the retriever. Additionally we'll compute the MRR (Mean Reciprocal Rank) metric. The MRR is the average of the reciprocal ranks of the first relevant document. For example, if we retrieve 5 documents (k=5) and the relevant document is ranked 2nd, the reciprocal rank would be 1/2. We calculate the reciprocal rank for each question and then take the average to get the MRR.

In [ ]:
test_data = (data_path / "prepared_data_test.jsonl").read_text().splitlines()
retriever_evaluation_data = []

# we only need the ref_doc_id and question from the test data

for record in test_data:
    json_record = json.loads(record)
    retriever_evaluation_data.append({"ref_doc_id":json_record["ref_doc_id"], "question":json_record["question"]})

In [ ]:
if mlflow_available:
    pre_signed_url = mlflow_utils.create_presigned_url(mlflow_config["tracking_server_name"])
    display(Markdown(f"Our experiment results will be logged to MLFlow. You can view them from the [MLFlow UI]({pre_signed_url})") )

In [ ]:
k = 3 # number of documents to retrieve
faiss_retriever = vec_db.as_retriever(search_kwargs={"k": k})


correct = 0
reciprocal_rank = 0
num_examples = 400 # Number of examples to evaluate
for i, eval_data in enumerate(retriever_evaluation_data[:num_examples]):
    returned_docs = faiss_retriever.invoke(eval_data["question"])
    returned_doc_ids = [doc.metadata["unique_id"] for doc in returned_docs]
    if eval_data["ref_doc_id"] in returned_doc_ids:
        correct += 1
        reciprocal_rank += 1 / (returned_doc_ids.index(eval_data["ref_doc_id"]) + 1)
    else:
        continue

hit_rate = correct / num_examples
mrr = reciprocal_rank / num_examples

print(f"Hit rate @k={k}: {hit_rate}")
print(f"MRR @k={k}: {mrr}")

if mlflow_available:
    mlflow.set_experiment("Retriever Evaluation")
    with mlflow.start_run(run_name="baseline_retriever"):
        mlflow.log_param("retriever", "FAISS")
        mlflow.log_param("k", k)
        mlflow.log_metric("hit_rate", hit_rate)
        mlflow.log_metric("mrr", mrr)

The evaluation results above may vary but we should see a hit rate of over 0.92 and an MRR of over 0.85. These results are quite good and indicate that the retriever is able to find the relevant context for most questions. If this was not the case, then using a different embedding model or fine-tuning the retriever would be possible options to consider. A number of libraries exist that can be used to fine-tune or train a custom embedding model for retrieval including:
- [sentence-transformers](https://www.sbert.net/docs/sentence_transformer/training_overview.html)
- [RAGatouille](https://github.com/bclavie/RAGatouille)

There are other ways to improve the retriever performance such as using hybrid search that combines both dense and sparse retrieval methods. 

For example below, we can improve the performance of the above retriever by ensembling it with a sparse retriever like BM25. This tends to work well with domain specific datasets as it combines the strengths of keyword search with semantic search. We'll use langchain's [EnsembleRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/ensemble/) to combine the dense retriever with BM25. However many vector dbs offer hybrid search capabilities out of the box such as  [OpenSearch](https://opensearch.org/docs/latest/search-plugins/hybrid-search/).


In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
bm_25 = BM25Retriever.from_documents(documents)
bm_25.k = k


ensemble_retriever = EnsembleRetriever(
    retrievers=[faiss_retriever, bm_25], weights=[0.75, 0.25] # you can fine-tune the weights here
)

correct = 0
average_rank = 0
num_examples = 400 # Number of examples to evaluate
for i, eval_data in enumerate(retriever_evaluation_data[:num_examples]):
    returned_docs = ensemble_retriever.invoke(eval_data["question"])
    returned_doc_ids = [doc.metadata["unique_id"] for doc in returned_docs]
    if eval_data["ref_doc_id"] in returned_doc_ids:
        correct += 1
        average_rank += 1 / (returned_doc_ids.index(eval_data["ref_doc_id"]) + 1)
    else:
        continue

hit_rate = correct / num_examples
mrr = average_rank / num_examples

print(f"Hit rate with Hybrid Search @k={k}: {hit_rate}")
print(f"MRR with Hybrid Search @k={k}: {mrr}")

if mlflow_available:
    mlflow.set_experiment("Retriever Evaluation")
    with mlflow.start_run(run_name="hybrid_retriever"):
        mlflow.log_param("k", k)
        mlflow.log_param("retriever", "hybrid")
        mlflow.log_param("weights", ensemble_retriever.weights)
        mlflow.log_metric("hit_rate", hit_rate)
        mlflow.log_metric("mrr", mrr)

You should see an improvement in the hit rate and MRR after ensembling with BM25.

### Build the Retrieval Augmented Generation (RAG) pipeline
Now that we are satisfied that the retriever is performing reasonably well, we can move on to the generation step. We'll build a basic Chain that given a question will retrieve the relevant context and invoke a Large Language Model to generate the answer. We will use the smaller `mistral.mistral-7b-instruct-v0:2` to generate the responses, this will also be the model that we will fine-tune in the subsequent notebooks.

In [ ]:
from langchain_aws.chat_models import ChatBedrockConverse
from langchain_core.prompts import ChatPromptTemplate

llm_modelId = "mistral.mistral-7b-instruct-v0:2"

llm = ChatBedrockConverse(
    model_id=llm_modelId, max_tokens=1000, temperature=0,
    client=bedrock_runtime,
)


Below is the prompt template that will be used to generate the answer. It's a simple template that will provide basic single-turn functionality and not include any guardrails to constrain the interaction. This is a good starting point but in production scenarios, you would want to add more sophisticated guardrails to ensure the model generates safe and accurate responses.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from operator import itemgetter

template = """You are a Banking Regulatory Compliance expert. You have been asked to provide guidance on the following question using the referenced regulations below.
If the referenced regulations do not provide an answer, indicate to the user that you are unable to provide an answer and suggest they consult with a legal expert.

----------------------
{context}
----------------------

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": ensemble_retriever, "question": RunnablePassthrough()}
)

# produce an output that contains the answer and the context that was passed to the model
generate_answer = {"answer": prompt | llm | output_parser,
                   "context": itemgetter("context")}

chain = setup_and_retrieval | generate_answer

Let's invoke the chain with a sample test question and examine the results.

In [ ]:
sample_record = json.loads(test_data[10])
sample_question = sample_record["question"]
sample_answer = sample_record["answer"]
rprint(f"[bold green]Sample question:[/bold green] {sample_question}")
response = chain.invoke(sample_question)
generated_answer = response["answer"]
rprint(f"[bold green]Generated answer:[/bold green] {generated_answer}")
rprint(f"[bold green]Ground truth answer:[/bold green] {sample_answer}")

### RAG Evaluation
While a manual examination of the generated answers is one of the most reliable ways to evaluate the model, it may not always be the most scalable especially as we iterate on the pipeline. There are various frameworks for evaluating RAG pipelines. For simplicity, we will look at two metrics that are readily available with [Bedrock Guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails.html). Specifically, Bedrock guardrails provide two [Contextual Grounding](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-contextual-grounding-check.html)
- **Grounding** – This checks if the model response is factually accurate based on the source and is grounded in the source. Any new information introduced in the response will be considered un-grounded.
- **Relevance** – This checks if the model response is relevant to the user query.
For example a response that is factually accurate but not relevant to the user query will be considered irrelevant. Similarly a response that is relevant but not factually accurate based on teh provided context will be considered ungrounded.

We'll will first compute the average scores for the ground truth answers and then compare them with the scores generated from RAG pipeline.

In [ ]:
# the guardrail should be created as part of the workshop
# if not you can create "rag_eval" guardrail in the console with only the contextual grounding check enabled
eval_guardrail = [gr for gr in bedrock_client.list_guardrails()["guardrails"] if gr["name"]=="rag_eval"]
if len(eval_guardrail) == 0:
    rprint("No RAG evaluation guardrail found. Please create one in the Bedrock console.")
else:
    eval_guardrail = eval_guardrail[0]
eval_guardrail_id = eval_guardrail["id"]

In [ ]:
def invoke_rag_eval_guardrail(guardrail_id, question, context, response):

    guardrail_payload = [
        {
            "text": {
                "text": context,
                "qualifiers": ["grounding_source"],
            }
        },
        {"text": {"text": question, "qualifiers": ["query"]}},
        {"text": {"text": response}},
    ]

    response = bedrock_runtime.apply_guardrail(
        guardrailIdentifier=guardrail_id,
        guardrailVersion="1",
        source="OUTPUT",
        content=guardrail_payload,
    )
    assessments = response["assessments"][0]["contextualGroundingPolicy"]["filters"]
    grounding_score = [
        metric for metric in assessments if metric["type"] == "GROUNDING"
    ][0]["score"]
    relevance_score = [
        metric for metric in assessments if metric["type"] == "RELEVANCE"
    ][0]["score"]
    return {"grounding_score": grounding_score, "relevance_score": relevance_score}

In [ ]:
# test on a sample question
invoke_rag_eval_guardrail(eval_guardrail_id, sample_question, sample_record["context"], generated_answer)

In [ ]:
async def generate_answer_async(rag_chain, example):
    """Helper function to generate an answer asynchronously"""
    example = json.loads(example)
    response = await rag_chain.ainvoke(example["question"])
    contexts = [doc.page_content for doc in response["context"]]
    row = {"question": example["question"], "answer": response["answer"], "contexts": contexts, "ground_truth": example["answer"]}
    return row

In [ ]:
# we'll just use the first 100 examples for evaluation
NUM_SAMPLE_LLM_EVALUATION = 100
eval_rows = []
for example in test_data[:NUM_SAMPLE_LLM_EVALUATION]:
    eval_rows.append(generate_answer_async(chain, example))
event_loop = asyncio.get_event_loop()
eval_data= event_loop.run_until_complete(asyncio.gather(*eval_rows))

In [ ]:
def evaluate_rag_guardrail(guardrail_id, questions, contexts, answers):
    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        for question, context, answer in zip(questions, contexts, answers):
            results.append(
                executor.submit(
                    invoke_rag_eval_guardrail, guardrail_id, question, "\n".join(context), answer
                )
            )
        
    eval_rows = [result.result() for result in results]
        
    grounding_scores = [row["grounding_score"] for row in eval_rows]
    relevance_scores = [row["relevance_score"] for row in eval_rows]
    grounding_score = sum(grounding_scores) / len(grounding_scores)
    relevance_score = sum(relevance_scores) / len(relevance_scores)
    return grounding_score, relevance_score

In [ ]:
rprint("Evaluating the ground truth answers using the RAG evaluation guardrail")
ground_truth_grounding_score, ground_truth_relevance_score = evaluate_rag_guardrail(
    eval_guardrail_id,
    [row["question"] for row in eval_data],
    [row["contexts"] for row in eval_data],
    [row["ground_truth"] for row in eval_data],
)


rprint("Evaluating the baseline RAG responses using the RAG evaluation guardrail")
baseline_grounding_score, baseline_relevance_score = evaluate_rag_guardrail(
    eval_guardrail_id,
    [row["question"] for row in eval_data],
    [row["contexts"] for row in eval_data],
    [row["answer"] for row in eval_data],
)

rprint(f"Ground truth grounding score: {ground_truth_grounding_score}\nBaseline grounding score: {baseline_grounding_score}\n")
rprint(f"Ground truth relevance score: {ground_truth_relevance_score}\nBaseline relevance score: {baseline_relevance_score}")


In [ ]:
# save the evaluation results to a file
with open("base_evaluation.json", "w") as f:
    metrics = {
        "ground_truth": {
            "grounding_score": ground_truth_grounding_score,
            "relevancy": ground_truth_relevance_score,
        },
        
        "baseline": {
            "grounding_score": baseline_grounding_score,
            "relevancy": baseline_relevance_score,
        }
    }
    json.dump(metrics, f)
    
# log the evaluation results to MLFlow if available
if mlflow_available:
    mlflow.set_experiment("Banking Regulations RAG Evaluation")
    with mlflow.start_run(run_name="ground_truth"):
        mlflow.log_metric("grounding_score", ground_truth_grounding_score)
        mlflow.log_metric("relevance_score", ground_truth_relevance_score)
        
    with mlflow.start_run(run_name="baseline"):
        mlflow.log_metric("grounding_score", baseline_grounding_score)
        mlflow.log_metric("relevance_score", baseline_relevance_score)

### Conclusion
In this notebook, we have demonstrated how to use LangChain to build a hybrid search system that combines BM25 and FAISS retrievers to retrieve relevant documents for a given question. We have also shown how to use LangChain to generate answers to questions using a language model and evaluate the generated answers using Bedrock Contextual Guardrails.